# Reconstruct the dose from a treatment plan

This script calls `reconstruct_dose`, which
calculates the dose for all fields in a treatment plan.

To run this example, you will need:
- A DICOM RT file
- An external surface mesh, in STL or OBJ format
- Finite Pencil Beam kernel data

In [1]:
using Pkg; Pkg.activate(".")
using DoseCalculations

  Activating project at `~/Code/DoseCalculations.jl/examples`


## Setup

Load the DICOM plan

In [2]:
plan = load_dicom("path/to/dicom/RP.....dcm")

2-element Vector{VMATField{Float64, MultiLeafCollimatorSequence{Array{Float64, 3}, Vector{Float64}}}}:
 114, ϕg: -110.0°->110.0°, θb: 45.0°, MU: 1216.0

 114, ϕg: 110.0°->-110.0°, θb: 315.0°, MU: 1204.0


Load the external surface mesh, and create a cylindrical external surface:

In [3]:
mesh = load_structure_from_ply("path/to/body.stl")
trans = patient_to_fixed(getisocenter(plan[1]))

surf = CylindricalSurface(transform(mesh, trans));

Use the external surface to generate a set of dose points within the surface.

In [4]:
pos = DoseGridMasked(5., SurfaceBounds(surf), trans);

We'll setup the dose calculation algorithm, calibrating it at 100 MU = 1 Gy for 100cm² fieldsize at a source-axis distance of 1000m.

In [5]:
calc = FinitePencilBeamKernel("path/to/kernel/file.jld")
calibrate!(calc, 100., 100., 1000.);

## Dose Reconstruction

Now we can call `reconstruct_dose`.
This will iterate through every control point, computing dose for each aperture.

*Note:* If you're in the REPL, you can set `show_progress=true` which will render a progress bar.
It doesn't work in Jupyter Notebooks.

In [6]:
dose = reconstruct_dose(pos, surf, plan, calc; show_progress=false);

The dose can be stored for further analysis with other software:

In [ ]:
write_nrrd("dose.nrrd", pos, dose);